<a href="https://colab.research.google.com/github/146790g/TCGA-Breast-Cancer-Biomarker/blob/master/TypeA_Atrium_MRI_Unet_Pytorch_Training_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> **Data Set** : Atrium MRI

> **Model**: Unet

> **Coding** **Stye**: TypeA

> **Section**: Training

> **SubSection**: continued training , 02

In [1]:
%pwd
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install imgaug==0.4.0
!pip install nibabel==3.2.1

### ランタイムを再実行ボタンを押す

In [3]:
# パッケージのimport
import torch
import torch.nn as nn
import torch.nn.functional as F
import PIL
import cv2
import tensorflow as tf
print(tf.__version__)

import keras
print(keras.__version__)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset

from pathlib import Path
import glob
import torch
import numpy as np
import imgaug
import imageio
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

# パッケージのimport
import random
import math
import time
import pandas as pd
import numpy as np

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

import os.path as osp
from PIL import Image

import torch.utils.data as data


# パッケージのimport
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import os
import torch
#import torch.utils.data as data
from pathlib import Path


%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import PIL
import cv2

# パッケージのimport
import random
import math
import time
import pandas as pd
import numpy as np
import pickle
import datetime
import pytz


import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

import os.path as osp
from PIL import Image

import torch.utils.data as data


# パッケージのimport
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import os
import torch
#import torch.utils.data as data
from pathlib import Path


%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset

2.8.0
2.8.0


In [4]:
print(cv2.__version__)
print(PIL.__version__)
print(torch.__version__)
print(tf.__version__)
print(keras.__version__)

4.1.2
7.1.2
1.10.0+cu111
2.8.0
2.8.0


In [5]:
# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

使用デバイス： cuda:0


In [6]:
root_dir = Path('/content/gdrive/My Drive/Colab Notebooks/BBB Pytorch Lightning Udemy/06 Atrium Segmentation')
data_dir = root_dir/Path('Preprocessed')
metrics_dir = root_dir/Path('metrics')
network_dir = root_dir/Path('network')

import os
os.chdir(root_dir)
print(os.getcwd())

/content/gdrive/My Drive/Colab Notebooks/BBB Pytorch Lightning Udemy/06 Atrium Segmentation


#Dataloaderからバッチデータを作成

In [7]:
from torch.utils.data import Dataset
from dsets.dsets import ImageTransform
import random
from dsets.dsets import MakeDataset
from dsets.dsets import make_path_list
from dsets.dsets import MakeDataset_wo_ImageTransform
import torch.utils.data as data

train_list = make_path_list(data_dir,phase="train")
val_list = make_path_list(data_dir,phase="val")

file_list={'train':train_list,'val':val_list}

train_dataset = MakeDataset(file_list=file_list,transform=ImageTransform,phase='train')
val_dataset = MakeDataset(file_list=file_list,phase='val')

batch_size = 10

train_dl = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dl = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dl, "val": val_dl}


/usr/local/lib/python3.7/dist-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `SegmentationMapOnImage()` is deprecated. Use `SegmentationMapsOnImage` instead. (Note the plural 'Maps' instead of old 'Map'.).
  warn_deprecated(msg, stacklevel=3)
/usr/local/lib/python3.7/dist-packages/imgaug/imgaug.py:106: DeprecationWarning: Providing nb_classes to SegmentationMapsOnImage is no longer necessary and hence deprecated. The argument is ignored and can be safely removed.
  warn(msg, category=DeprecationWarning, stacklevel=stacklevel)


In [8]:
print(len(dataloaders_dict['train'].dataset))
print(len(dataloaders_dict['train']))
2271%4

2271
228


3

In [9]:
print(len(dataloaders_dict['val'].dataset))
print(len(dataloaders_dict['val']))

2271
228


#Unet作成

In [10]:

import torch
from torch.nn import init
import functools
from torch.autograd import Variable
from torch.optim import lr_scheduler
import numpy as np
from torchvision import models
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Function
import torch.utils.data as data
import torch



class DoubleConv(torch.nn.Module):
    """
    Helper Class which implements the intermediate Convolutions
    """
    def __init__(self, in_channels, out_channels):
        
        super().__init__()
        self.step = torch.nn.Sequential(torch.nn.Conv2d(in_channels, out_channels, 3, padding=1),
                                        torch.nn.ReLU(),
                                        torch.nn.Conv2d(out_channels, out_channels, 3, padding=1),
                                        torch.nn.ReLU())
        
    def forward(self, X):
        return self.step(X)


class UNet(torch.nn.Module):
    def __init__(self,n_classes):
        super().__init__()
        
        self.layer1 = DoubleConv(1, 64)
        self.layer2 = DoubleConv(64, 128)
        self.layer3 = DoubleConv(128, 256)
        self.layer4 = DoubleConv(256, 512)
        
        self.layer5 = DoubleConv(512+256, 256)
        self.layer6 = DoubleConv(256+128, 128)
        self.layer7 = DoubleConv(128+64, 64)
        self.layer8 = torch.nn.Conv2d(64,n_classes,1)
        
        self.maxpool = torch.nn.MaxPool2d(2)
        
    def forward(self, x):
        
        x1 = self.layer1(x)
        x1m = self.maxpool(x1)
        
        x2 = self.layer2(x1m)
        x2m = self.maxpool(x2)
        
        x3 = self.layer3(x2m)
        x3m = self.maxpool(x3)
        
        x4 = self.layer4(x3m)
        
        x5 = torch.nn.Upsample(scale_factor=2, mode="bilinear")(x4)
        x5 = torch.cat([x5, x3], dim=1)
        x5 = self.layer5(x5)
        
        x6 = torch.nn.Upsample(scale_factor=2, mode="bilinear")(x5)
        x6 = torch.cat([x6, x2], dim=1)
        x6 = self.layer6(x6)
        
        x7 = torch.nn.Upsample(scale_factor=2, mode="bilinear")(x6)
        x7 = torch.cat([x7, x1], dim=1)
        x7 = self.layer7(x7)
        
        ret = self.layer8(x7)
        return ret
        


class DiceLoss3(nn.Module):
    """
    class to compute the Dice Loss
    """
    def __init__(self,label):
        super().__init__()
        self.label = label

    def forward(self,outputs):
                
        # Flatten label and prediction tensors
        preds=torch.argmax(outputs,dim=1)

        posLabel_mask = self.label>0
        negLabel_mask = ~posLabel_mask
        posPred_mask = preds>0
        negPred_mask = ~posPred_mask

        pos_count = posLabel_mask.sum().to(torch.float64)
        neg_count = negLabel_mask.sum().to(torch.float64)

        trueNeg_count =  (negLabel_mask & negPred_mask).sum().to(torch.float64)
        truePos_count =  (posLabel_mask & posPred_mask).sum().to(torch.float64)
        falsePos_count = neg_count - trueNeg_count
        falseNeg_count = pos_count - truePos_count

        N=truePos_count+falsePos_count+falseNeg_count+ 1e-8

        #dice loss
        dice = (2*truePos_count)/N
        loss = 1- dice
        

        #accuracy 
        accuracy = (truePos_count + trueNeg_count)/ (pos_count + neg_count + 1e-8)
        #precision
        precision = truePos_count/ (truePos_count + falsePos_count + 1e-8)
        #recall
        recall = truePos_count/ (truePos_count + falseNeg_count + 1e-8)

        return accuracy, precision, recall, loss

In [11]:
# モデルの定義
model = UNet(n_classes=2)


In [12]:
for n, p in model.named_parameters():
    print(n)
    #print(p)
    print("--------")

layer1.step.0.weight
--------
layer1.step.0.bias
--------
layer1.step.2.weight
--------
layer1.step.2.bias
--------
layer2.step.0.weight
--------
layer2.step.0.bias
--------
layer2.step.2.weight
--------
layer2.step.2.bias
--------
layer3.step.0.weight
--------
layer3.step.0.bias
--------
layer3.step.2.weight
--------
layer3.step.2.bias
--------
layer4.step.0.weight
--------
layer4.step.0.bias
--------
layer4.step.2.weight
--------
layer4.step.2.bias
--------
layer5.step.0.weight
--------
layer5.step.0.bias
--------
layer5.step.2.weight
--------
layer5.step.2.bias
--------
layer6.step.0.weight
--------
layer6.step.0.bias
--------
layer6.step.2.weight
--------
layer6.step.2.bias
--------
layer7.step.0.weight
--------
layer7.step.0.bias
--------
layer7.step.2.weight
--------
layer7.step.2.bias
--------
layer8.weight
--------
layer8.bias
--------


In [13]:
# 学習させるパラメータ以外は勾配計算をなくし、変化しないように設定
for name, param in model.named_parameters():
  param.requires_grad = True

##logMetrics関数の作成

In [14]:
METRICS_ACCURACY_NDX=0
METRICS_PRECISION_NDX=1
METRICS_RECALL_NDX=2
METRICS_LOSS_NDX=3

METRICS_SIZE = 4

def logMetrics(epoch_ndx,phase,metrics_t):
  
  metrics_dict['phase'].append(phase)
  metrics_dict['epoch_ndx'].append(epoch_ndx)
  metrics_dict['accuracy'].append(metrics_t[METRICS_ACCURACY_NDX].mean())
  metrics_dict['precision'].append(metrics_t[METRICS_PRECISION_NDX].mean())
  metrics_dict['recall'].append(metrics_t[METRICS_RECALL_NDX].mean())
  metrics_dict['loss'].append(metrics_t[METRICS_LOSS_NDX].mean())

  print('{}: Loss: {:.6f} Accuracy: {:.6f} Precision: {:.6f} Recall: {:.6f}'.format(phase,
                                                                                    metrics_t[METRICS_ACCURACY_NDX].mean(),
                                                                                    metrics_t[METRICS_PRECISION_NDX].mean(),
                                                                                    metrics_t[METRICS_RECALL_NDX].mean(),
                                                                                    metrics_t[METRICS_LOSS_NDX].mean()))

#学習済みモデルおよび評価指標の保存用の関数の作成

In [15]:
def save_file(metrics,epoch_ndx):
  save_filename = 'logMetrics2_%s.pickle' % (epoch_ndx)
  save_path = os.path.join(metrics_dir,save_filename)
  with open(save_path,mode='wb') as f:
    pickle.dump(metrics,f)

def save_network(network, epoch_ndx):
  save_filename = 'net2_%s.pth' % (epoch_ndx)
  save_path = os.path.join(network_dir, save_filename)
  torch.save(network.cpu().state_dict(), save_path)

#train_model関数の作成

In [16]:
print(device)

cuda:0


In [17]:
metrics_dict = {'epoch_ndx':[],'phase':[],'accuracy':[],'precision':[],'recall':[],'loss':[]}

def train_model(model, dataloaders_dict,epoch_start,epoch_end,save_freq):

  now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
  print('start time:',now)

  t_epoch_start = time.time()
  optimizer = optim.SGD(model.parameters(), lr=1e-3,momentum=0.9, weight_decay=5e-4)
  num_epochs=epoch_end-epoch_start+1

  for epoch_ndx in range(epoch_start,epoch_end+1):
    t_epoch_start = time.time()

    print('-------------------------------------------------------------------------------------')
    print('Epoch {} 【From Epoch {}:To Epoch {}】:Now {}'.format(epoch_ndx,epoch_start,epoch_end,now))
    print('-------------------------------------------------------------------------------------')
    
    model.to(device)

    for phase in ['train', 'val']:

      if phase == 'train':
        model.train()  # モデルを訓練モードに
      else:
        model.eval()   # モデルを検証モードに

      metrics_g = torch.zeros(METRICS_SIZE,len(dataloaders_dict[phase]),device=device,)

      # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
      if (epoch_ndx == 0) and (phase == 'train'):
        continue

      for batch_ndx, batch_tup in enumerate(tqdm(dataloaders_dict[phase])):
        input_t, label_t = batch_tup
        input_g=input_t.to(device)
        label_g=label_t.to(device)
        #del input_t,label_t,batch_tup

        # optimizerを初期化
        optimizer.zero_grad()

      # 順伝搬（forward）計算
        with torch.set_grad_enabled(phase == 'train'):
        
          outputs = model(input_g)
          criterion = DiceLoss3(label_g.long())
          accuracy, precision, recall, loss =criterion(outputs)

          metrics_g[METRICS_ACCURACY_NDX,batch_ndx] = accuracy.detach()
          metrics_g[METRICS_PRECISION_NDX,batch_ndx] = precision.detach()
          metrics_g[METRICS_RECALL_NDX,batch_ndx] = recall.detach()
          metrics_g[METRICS_LOSS_NDX,batch_ndx] = loss.detach()

          # 訓練時はバックプロパゲーション
          if phase == 'train':
            loss.backward()
            optimizer.step()
    
      metrics_t=metrics_g.to('cpu')
      logMetrics(epoch_ndx,phase,metrics_t) 

    if (epoch_ndx % save_freq ==0) & (epoch_ndx>0):
      print('saving the model at the end of epoch %d' % (epoch_ndx))
      save_network(model,epoch_ndx)
      print('saving the logMetrics at the end of epoch %d' % (epoch_ndx))
      save_file(metrics_dict,epoch_ndx)      

    t_epoch_finish = time.time()
    print('Epoch:{} timer:  {:.4f} sec.'.format(epoch_ndx,t_epoch_finish - t_epoch_start)) 


##①初回学習の実行

### epoch_ndx=0,1,2,3,4,5について学習する
###このうち、epoch_ndx=5 については、学習済みモデル及び評価指標を保存する
###初回学習のときには、epoch_start=0とする

In [18]:
# 学習・検証を実行する
epoch_start=0
save_freq=5
epoch_end=10

train_model(model, dataloaders_dict,epoch_start,epoch_end,save_freq)

start time: 2022-04-20 13:18:54.222704+09:00
-------------------------------------------------------------------------------------
Epoch 0 【From Epoch 0:To Epoch 10】:Now 2022-04-20 13:18:54.222704+09:00
-------------------------------------------------------------------------------------


  0%|          | 0/228 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
100%|██████████| 228/228 [00:22<00:00, 10.21it/s]


val: Loss: 0.993726 Accuracy: 0.000000 Precision: 0.000000 Recall: 1.000000
Epoch:0 timer:  26.1374 sec.
-------------------------------------------------------------------------------------
Epoch 1 【From Epoch 0:To Epoch 10】:Now 2022-04-20 13:18:54.222704+09:00
-------------------------------------------------------------------------------------


  0%|          | 0/228 [00:00<?, ?it/s]


RuntimeError: ignored

#END